# HUGS - HUb for Greenhouse gas data Science

## Overview

HUGS works in a modular way without a fixed hierarchical structure. 

There are multiple modules

* Datasources
* Instrument
* Sites
* Networks

For example they can be arranged as

* Network
    * Site
        * Instrument
            * Datasource
            * Datasource
            * Datasource

Or
            

* Network
    * Datasource
    * Datasource
    * Datasource

There is no set fixed hierarchy for these modules
        




Data is stored within objects and these objects may hold links to data in the object store.

In the following code we will read in a data file, analyse the data contained within it, segment the data into sections which can be easily stored in the object store and then recombine these dataframes ready for export to the end user.

In [ ]:
# Suppress some Pandas warnings - these will be fixed
import warnings
warnings.filterwarnings('ignore')
# Data pretty printer for nicer printing of data
import pprint
# User PrettyPrinter to print them in a nicer way
pp = pprint.PrettyPrinter(indent=2)

In [ ]:
# For listing of objects in the object store
from objectstore.hugs_objstore import list_object_names
# To get the local bucket (a container for data in the object store)
from objectstore.local_bucket import get_local_bucket
# The object to process and store CRDS data
from processing._crds import CRDS

## Processing data

Here we read in a data file from the Bilsdale site using the read_file() function from the CRDS class

This function
* Creates a CRDS object
* Collects metadata
* Splits the data into separate dataframes for each gas
* Creates a Datasource object for the gas, holding data and metadata
* Stores this data within the CRDS object



In [ ]:
filename = "data/bsd.picarro.1minute.248m.dat"

In [ ]:
crds = CRDS.read_file(filename)

We can now check the daterange for the data read in

In [ ]:
crds.get_daterange()

To check that the function has processed the datafile correctly and picked up the right dates I've used the Linux `head` and `tail` applications to get the first and last rows (not containing NaNs) from the data file.

Head

`140130 105230       air    8   1960.24   0.236    26    409.66   0.028    26    204.62   6.232    26`

Tail

`140130 142030       air    8   1952.24   0.674    25    408.78   0.019    25    196.35   6.879    25`


We can view some of the data stored in each Datasource within this CRDS object

In [ ]:
datasources = crds.get_datasources()

for d in datasources:
    data = d.get_data()
    # Print the top two lines of each dataframe
    print("\n", data.head(2))

## Metadata

We can also check the metadata collected. This is stored within a Metadata object within the CRDS object and we can now view this.

In [ ]:
metadata = crds.get_metadata()
pp.pprint(metadata)


## Visualisation

We can now visualize some of this data



In [ ]:
%matplotlib notebook

# Get the first datasource
datasource = datasources[0]
for datasource in datasources:
    data = datasource.get_data()
    # Get column names
    col_names = list(data.columns)
    ax = data.plot(x="Datetime", y=col_names[1], elinewidth=1, linewidth=0, marker="o", markersize=3, 
                   yerr=col_names[2], legend=None, color="#59a14f", title=datasource._name.upper())
    ax.set(xlabel="Time", ylabel="Count")




## Object Store

The processed and segmented data is now ready to be stored in the object store.

The data for each gas is stored within a Pandas DataFrame that gets converted into a compressed binary HDF5 file format similar to that used by NetCDF. Each datasource stores its data with a key containing its own universally unique identifier (UUID).

Each object has a .save() function that saves the object to the data store


In [ ]:
# Get a bucket, a container used to store data in the object store
bucket = get_local_bucket(empty=True)
# Save the object to the object store
crds.save(bucket)

The code for this save function is given below

```py
def save(self, bucket=None):
    """ Save the object to the object store

        Args:
            bucket (dict, default=None): Bucket for data
        Returns:
            None
    """
    if self.is_null():
        return

    # If a bucket isn't passed, get the hugs bucket
    if bucket is None:
        bucket = _get_bucket()
    
    # Create the key at which to store this object
    crds_key = "%s/uuid/%s" % (CRDS._crds_root, self._uuid)
    
    # Get the datasources to save themselves to the object store
    for d in self._datasources:
        d.save(bucket)
    
    # Save this object as JSON
    _ObjectStore.set_object_from_json(bucket=bucket, key=crds_key, data=self.to_data())
```

This function then saves the CRDS object and each of its Datasources to the object store. 
As each Datasource holds gas data this in turn is saved by the save function of each Datasource.

We can now view the structure of the object store by querying the keys in the bucket

In [ ]:
# List all the objects in the container
bucket_list = list_object_names(bucket)

pp.pprint(bucket_list)


### Keys

Each object in the object store is saved at a key. This allows each object to be stored at a unique location.

Objects are stored as so

`{object_name}/uuid/{uuid}`

Objects that contain dates are given the key

`{object_name}/uuid/{uuid}/{startdatetime_enddateime}`

For example data stored in a Datasource at key

`datasource/uuid/8338eb59-8eac-4b6c-8331-61d68e0e2ed3`

is given the key

`data/uuid/8338eb59-8eac-4b6c-8331-61d68e0e2ed3/2014-01-30T10:52:30_2014-01-30T14:20:30`

Notice that the UUIDs in both keys are the same.

Some objects can also be accessed by name through their name key

`{object_name}/name/{name}/{uuid}`

This allows lookup of an object's UUID by its name

## Searching for data

We can look for data in the object store by date

In [ ]:
from processing._crds import CRDS

# Get the search start date
# As this datafile only has data for a single day we just use the same start and end datetime
start = CRDS.to_datetime("2014-01-30")
end =  CRDS.to_datetime("2014-01-30")

object_type = "datasource"
# We can now search the object store for keys
keys = crds.search_store(bucket=bucket, root_path=object_type, datetime_begin=start, datetime_end=end)

pp.pprint(keys)

These are the keys for data in the object store that holds data between those dates. Currently as datafiles are not large they are not being split into week/month segments depending on the resolution of the readings. This feature will be implemented soon.

Now that we have the keys for the Datasources containing this data we can recombine these pieces into a single 
dataframe. Here we will choose to use all three.

Functions to select and order data within the produced dataframe will be implemented.


In [ ]:
from processing._recombination import get_sections

# Get the data at each of the found keys
datasources = get_sections(bucket, keys)

print(datasources)

In [ ]:
dataframes = [datasource.get_data() for datasource in datasources]

for d in dataframes:
    print("\n",d.head(2))

In [ ]:
from processing._recombination import combine_sections

# Combine each of the sections into a single dataframe
combined = combine_sections(dataframes)

print(combined)

We can now have a look at this data again and make sure it comes out correctly

In [ ]:
col_names = list(combined.columns)

print(col_names)

In [ ]:
ax = combined.plot(x="Datetime", y="ch4 count", elinewidth=1, linewidth=0, marker="o", markersize=3, 
               yerr="ch4 stdev", legend=None, color="#4e79a7", title="CH4")

ax = combined.plot(x="Datetime", y="co2 count", elinewidth=1, linewidth=0, marker="o", markersize=3, 
               yerr="co2 stdev", legend=None, color="#59a14f", title="CO2")

ax = combined.plot(x="Datetime", y="co count", elinewidth=1, linewidth=0, marker="o", markersize=3, 
               yerr="co stdev", legend=None, color="#e15759", title="CO")

We've now seen data being processed, metadata extracted and saved in the object store. It was then recombined from the object store and exported as a single dataframe.